In [1]:
!mkdir -p ../datasets && cd ../datasets && if [ ! -f "input.txt" ]; then curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -o input.txt; fi && cd ../jupyter

In [2]:
# read it in to inspect it
with open('../datasets/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(f'length of dataset in characters: {len(text):,}')

length of dataset in characters: 1,115,394


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# here are all the unique characters that occur in this text file
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [5]:
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create a mapping from charactors to integers
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [7]:
print(encode("hi! there"))
print(decode(encode("hi! there")))

[46, 47, 2, 1, 58, 46, 43, 56, 43]
hi! there


In [8]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)

In [9]:
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [10]:
# Let's now split up the data into train and validation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target  = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [13]:
torch.manual_seed(1337)
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?

In [14]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [15]:
xb, yb = get_batch('train')

In [16]:
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 

In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directory reads of the logits for the net token from a lookup table
        self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=vocab_size)

    def forward(self, idx, targets):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx)
        return logits


m = BigramLanguageModel(vocab_size)
out = m(xb, yb)
print(out.shape)

torch.Size([4, 8, 65])


`self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)`は、**各単語（トークン）に対する「次にどの単語が来るか」の予測スコア（ロジット）を格納するルックアップテーブル**を作成する役割を担っています。

このBigramモデルの文脈で、もう少し具体的に解説します。

---

### ## `nn.Embedding`の基本

`nn.Embedding`は、PyTorchにおいて単語のような離散的なIDを、密なベクトル表現（埋め込みベクトル）に変換するためのレイヤーです。基本的には巨大な行列であり、**単語のIDをインデックスとして、対応する行ベクトルを抜き出す**という動作をします。

`nn.Embedding(num_embeddings, embedding_dim)` の引数は以下の通りです。
* `num_embeddings`: 埋め込む単語の総数。通常は語彙サイズ (`vocab_size`) を指定します。
* `embedding_dim`: 各単語を表現するベクトルの次元数。

---

### ## このモデルにおける`nn.Embedding`の特別な役割

この`BigramLanguageModel`では、`embedding_dim`に`vocab_size`が指定されています。これが重要なポイントです。

`self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)`

これは、サイズが `(vocab_size, vocab_size)` の行列を作成することを意味します。

* **行 (Rows)**: `vocab_size`個の行は、語彙に含まれる各トークンに対応します。例えば、0番目の行はIDが0のトークン、1番目の行はIDが1のトークン...という具合です。
* **列 (Columns)**: `vocab_size`個の列を持つベクトル（各行）が、**次にどのトークンが来るかの予測スコア（ロジット）**を表します。

#### ### 処理の流れ

1.  **入力**: `forward`メソッドに、トークンIDのシーケンス `idx` (shape: `(B, T)`、Bはバッチサイズ, Tはシーケンス長) が入力されます。
2.  **ルックアップ**: `self.token_embedding_table(idx)` が実行されると、`idx` の各トークンIDに対応する行ベクトルがテーブルから抜き出されます。
3.  **出力 (`logits`)**: 出力される `logits` のshapeは `(B, T, vocab_size)` となります。これは、入力シーケンスの各トークンに対して、「次に来る単語の候補（全`vocab_size`個）」それぞれの予測スコアを持っていることを意味します。



#### ### なぜこれがBigramモデルなのか？

**Bigramモデル**は、**直前の1つの単語だけを見て次の単語を予測**します。
このコードはまさにその考え方を実装しています。

* あるトークン（例: "hello"）が入力されると、`nn.Embedding`テーブルから "hello" に対応する行を引いてきます。
* その行ベクトル（サイズ `vocab_size`）が、次に "world" が来る確率、"there" が来る確率、"!" が来る確率...といった、語彙の全単語に対する予測スコア（ロジット）そのものになるわけです。

つまり、この `nn.Embedding` レイヤーは、単に単語をベクトル化するだけでなく、**「ある単語」から「次の単語の予測ロジット」へのマッピング**という、モデルの予測機能そのものを担っています。これがこのモデルの核心部分です。👍